# Build Docker Image

In this notebook, we will build the docker container that contains the  model, Flask web application, model driver and all dependencies. Make sure you have logged in using docker login.

In [1]:
import os
from os import path
import json
import shutil

In [2]:
os.makedirs('flaskwebapp', exist_ok=True)
os.makedirs(os.path.join('flaskwebapp', 'nginx'), exist_ok=True)
os.makedirs(os.path.join('flaskwebapp', 'etc'), exist_ok=True)

In [3]:
shutil.copy('driver.py', 'flaskwebapp')
shutil.copy('duplicate_model.py', 'flaskwebapp')
shutil.copy('model.pkl', 'flaskwebapp')
shutil.copy('questions.tsv', 'flaskwebapp')
shutil.copy('ItemSelector.py', 'flaskwebapp')
os.listdir('flaskwebapp')

['duplicate_model.py',
 'questions.tsv',
 'ItemSelector.py',
 'driver.py',
 'model.pkl',
 'etc',
 'nginx']

In [4]:
%%writefile flaskwebapp/app.py

from flask import Flask, request
import logging
import json
import driver

app = Flask(__name__)
predict_for = driver.get_model_api()
 
@app.route("/score", methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    request_input = request.json['input']
    response = predict_for(request_input)
    print(response)
    return json.dumps({'result': str(response)})


@app.route("/")
def healthy():
    return "Healthy"

# LightGBM Version
@app.route('/version', methods = ['GET'])
def version_request():
    return driver.version()

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

Writing flaskwebapp/app.py


In [5]:
%%writefile flaskwebapp/wsgi.py
from app import app as application

def create():
    print("Initialising")
    application.run(host='127.0.0.1', port=5000)

Writing flaskwebapp/wsgi.py


Here, we write the configuration for the Nginx which creates a proxy between ports 80 and 5000.

In [6]:
%%writefile flaskwebapp/nginx/app
server {
    listen 80;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Writing flaskwebapp/nginx/app


In [7]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Writing flaskwebapp/gunicorn_logging.conf


In [8]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()

Writing flaskwebapp/kill_supervisor.py


In [9]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Writing flaskwebapp/etc/supervisord.conf


We now create a custom image and install all the necessary dependencies.

In [10]:
%%writefile flaskwebapp/requirements.txt

pandas==0.23.3
click==6.7
configparser==3.5.0
Flask==0.12.2
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.18.4
# h5py==2.6.0

Writing flaskwebapp/requirements.txt


In [11]:
%%writefile flaskwebapp/dockerfile

FROM ubuntu:16.04

USER root
RUN mkdir /code
WORKDIR /code
RUN chmod -R a+w /code
ADD . /code/

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        wget \
        curl \
        git \
        supervisor \
        nginx && \
    rm -rf /var/lib/apt/lists/* &&\
    rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ 

ENV PYTHON_VERSION=3.5
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy pyyaml scipy \
    ipython jupyter ipykernel && \
    /opt/conda/bin/conda clean -ya
    
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
    
RUN pip install --upgrade pip && \
    pip install scikit-learn==0.19.1 && \
    pip install lightgbm==2.1.2 && \
    pip install -r /code/requirements.txt && \       
    /opt/conda/bin/conda clean -yt

EXPOSE 8888
EXPOSE 5000
EXPOSE 80

CMD ["supervisord", "-c", "/code/etc/supervisord.conf"]

Writing flaskwebapp/dockerfile


The image name below refers to our dockerhub account. If you wish to push the image to your account make sure you change the docker login.

In [12]:
docker_login = 'fboylu'
image_name = docker_login + '/mlaksdep'
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

In [13]:
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

Sending build context to Docker daemon  12.78MB
Step 1/16 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu

e2d282dc: Pulling fs layer 
f5e4b3b2: Pulling fs layer 
29f854da: Pulling fs layer 
33abe16a: Pulling fs layer 
2a16d85e: Pull complete  169B/169B9MBBDigest: sha256:14066a391d902c386d6164d44ade3460ba044abcdf8df88b0ff79a6f635be8d3
Status: Downloaded newer image for ubuntu:16.04
 ---> e13f3d529b1a
Step 2/16 : USER root
 ---> Running in b985abafbe6f
Removing intermediate container b985abafbe6f
 ---> 32f65530f977
Step 3/16 : RUN mkdir /code
 ---> Running in 3626f7f723e1
Removing intermediate container 3626f7f723e1
 ---> 358af063a38e
Step 4/16 : WORKDIR /code
Removing intermediate container bfdce76796e2
 ---> af987b816a58
Step 5/16 : RUN chmod -R a+w /code
 ---> Running in 1eb5fd21c858
Removing intermediate container 1eb5fd21c858
 ---> 47643259695a
Step 6/16 : ADD . /code/
 ---> 745abe94a228
Step 7/16 : RUN apt-get update && apt-get install -y --no-install-recommends         bui

Get:7 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libxml2 amd64 2.9.3+dfsg1-1ubuntu0.5 [697 kB]
Get:8 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libarchive13 amd64 3.1.2-11ubuntu0.16.04.3 [262 kB]
Get:9 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libkrb5support0 amd64 1.13.2+dfsg-5ubuntu2 [30.8 kB]
Get:10 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libk5crypto3 amd64 1.13.2+dfsg-5ubuntu2 [81.2 kB]
Get:11 http://archive.ubuntu.com/ubuntu xenial/main amd64 libkeyutils1 amd64 1.5.9-8ubuntu1 [9904 B]
Get:12 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libkrb5-3 amd64 1.13.2+dfsg-5ubuntu2 [273 kB]
Get:13 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgssapi-krb5-2 amd64 1.13.2+dfsg-5ubuntu2 [120 kB]
Get:14 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libidn11 amd64 1.32-3ubuntu1.2 [46.5 kB]
Get:15 http://archive.ubuntu.com/ubuntu xenial/main amd64 libgmp10 amd64 2:6.1.0+dfsg-2 [240 kB]
Ge

Get:82 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libcilkrts5 amd64 5.4.0-6ubuntu1~16.04.10 [40.1 kB]
Get:83 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libmpx0 amd64 5.4.0-6ubuntu1~16.04.10 [9764 B]
Get:84 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libquadmath0 amd64 5.4.0-6ubuntu1~16.04.10 [131 kB]
Get:85 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgcc-5-dev amd64 5.4.0-6ubuntu1~16.04.10 [2228 kB]
Get:86 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 gcc-5 amd64 5.4.0-6ubuntu1~16.04.10 [8426 kB]
Get:87 http://archive.ubuntu.com/ubuntu xenial/main amd64 gcc amd64 4:5.3.1-1ubuntu1 [5244 B]
Get:88 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libstdc++-5-dev amd64 5.4.0-6ubuntu1~16.04.10 [1426 kB]
Get:89 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 g++-5 amd64 5.4.0-6ubuntu1~16.04.10 [8319 kB]
Get:90 http://archive.ubuntu.com/ubuntu xenial/main amd64 g++ amd64 4:5.3.1-1ubuntu1 [1504 B

Unpacking libheimbase1-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected package libwind0-heimdal:amd64.
Preparing to unpack .../libwind0-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libwind0-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected package libhx509-5-heimdal:amd64.
Preparing to unpack .../libhx509-5-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libhx509-5-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected package libsqlite3-0:amd64.
Preparing to unpack .../libsqlite3-0_3.11.0-1ubuntu1_amd64.deb ...
Unpacking libsqlite3-0:amd64 (3.11.0-1ubuntu1) ...
Selecting previously unselected package libkrb5-26-heimdal:amd64.
Preparing to unpack .../libkrb5-26-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libkrb5-26-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unsele

Selecting previously unselected package linux-libc-dev:amd64.
Preparing to unpack .../linux-libc-dev_4.4.0-130.156_amd64.deb ...
Unpacking linux-libc-dev:amd64 (4.4.0-130.156) ...
Selecting previously unselected package libc6-dev:amd64.
Preparing to unpack .../libc6-dev_2.23-0ubuntu10_amd64.deb ...
Unpacking libc6-dev:amd64 (2.23-0ubuntu10) ...
Selecting previously unselected package libisl15:amd64.
Preparing to unpack .../libisl15_0.16.1-1_amd64.deb ...
Unpacking libisl15:amd64 (0.16.1-1) ...
Selecting previously unselected package cpp-5.
Preparing to unpack .../cpp-5_5.4.0-6ubuntu1~16.04.10_amd64.deb ...
Unpacking cpp-5 (5.4.0-6ubuntu1~16.04.10) ...
Selecting previously unselected package cpp.
Preparing to unpack .../cpp_4%3a5.3.1-1ubuntu1_amd64.deb ...
Unpacking cpp (4:5.3.1-1ubuntu1) ...
Selecting previously unselected package libcc1-0:amd64.
Preparing to unpack .../libcc1-0_5.4.0-6ubuntu1~16.04.10_amd64.deb ...
Unpacking libcc1-0:amd64 (5.4.0-6ubuntu1~16.04.10) ...
Selecting previ

...

installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11.0-py36h372c433_1 ...
installing: cffi-1.11.5-py36h9745a5d_0 ...
installing: setuptools-39.2.0-py36_0 .

jupyter_core-4.4.0   |   61 KB | ########## | 100% 
parso-0.2.1          |  119 KB | ########## | 100% 
wcwidth-0.1.7        |   25 KB | ########## | 100% 
gmp-6.1.2            |  744 KB | ########## | 100% 
certifi-2018.4.16    |  143 KB | ########## | 100% 
jpeg-9b              |  248 KB | ########## | 100% 
send2trash-1.5.0     |   16 KB | ########## | 100% 
ipython-6.4.0        |  1.0 MB | ########## | 100% 
nbformat-4.4.0       |  138 KB | ########## | 100% 
qt-5.9.6             | 86.7 MB | ########## | 100% 
hyperlink-18.0.0     |   63 KB | ########## | 100% 
mistune-0.8.3        |  268 KB | ########## | 100% 
jinja2-2.10          |  182 KB | ########## | 100% 
service_identity-17. |   17 KB | ########## | 100% 
twisted-18.4.0       |  4.9 MB | ########## | 100% 
jsonschema-2.6.0     |   63 KB | ########## | 100% 
blas-1.0             |    6 KB | ########## | 100% 
pandocfilters-1.4.2  |   12 KB | ########## | 100% 
setuptools-39.2.0    |  565 KB | ########## | 100% 
gstreamer-1.

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

#
# To activate this environment, use:
# > source activate py3.5
#
# To deactivate an active environment, use:
# > source deactivate
#

Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
openssl-1.0.2o-h20670df_0.tar.bz2            3.4 MB
attrs-18.1.0-py35_0.tar.bz2                   44 KB
certifi-2018.4.16-py35_0.tar.bz2             143 KB
automat-0.7.0-py35_0.tar.bz2                  52 KB
ruamel_yaml-0.15.37-py36h14c3975_2.tar.bz2     245 KB
cffi-1.11.5-py36h9745a5d_0.tar.bz2           212 KB
sqlite-3.24.0-h84994c4_0.tar.bz2             1.8 MB
readline-7.0-ha6073c6_4.tar.bz2              1.1 MB
asn1crypto-0.24.0-py36_0.tar.bz2             155 KB
pandocfilters-1.4.2-py35h1565a15_1.tar.bz2      12 KB
wcwidth-0.1.7-py35hcd08066_0.tar.bz2          25 KB
jsonschema-2.6.0-py35h4395190_0.tar.bz2       63 KB
urllib3-

Removed pandoc-2.2.1-h629c226_0.tar.bz2
Removed nbformat-4.4.0-py35h12e6e07_0.tar.bz2
Removed pysocks-1.6.8-py36_0.tar.bz2
Removed jupyter_console-5.2.0-py35h4044a63_1.tar.bz2
Removed prompt_toolkit-1.0.15-py35hc09de7a_0.tar.bz2
Removed html5lib-1.0.1-py35h2f9c1c0_0.tar.bz2
Removed bleach-2.1.3-py35_0.tar.bz2
Removed terminado-0.8.1-py35_1.tar.bz2
Removed idna-2.7-py35_0.tar.bz2
Removed setuptools-39.2.0-py35_0.tar.bz2
Removed notebook-5.6.0-py35_0.tar.bz2
Removed parso-0.2.1-py35_0.tar.bz2
Removed pycparser-2.18-py35h61b3040_1.tar.bz2
Removed pickleshare-0.7.4-py35hd57304d_0.tar.bz2
Removed pcre-8.42-h439df22_0.tar.bz2
Removed libedit-3.1.20170329-h6b74fdf_2.tar.bz2
Removed six-1.11.0-py35h423b573_1.tar.bz2
Removed requests-2.18.4-py36he2e5f8d_1.tar.bz2
Removed libgcc-ng-7.2.0-hdf63c60_3.tar.bz2
Removed qtconsole-4.3.1-py35h4626a06_0.tar.bz2
Removed python-3.5.5-hc3d631a_4.tar.bz2
Removed hyperlink-18.0.0-py35_0.tar.bz2
Removed constantly-15.1.0-py35_0.tar.bz2
Removed libgfortran-ng-7

  Running setup.py bdist_wheel for configparser: started
  Running setup.py bdist_wheel for configparser: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/a3/61/79/424ef897a2f3b14684a7de5d89e8600b460b89663e6ce9d17c
  Running setup.py bdist_wheel for json-logging-py: started
  Running setup.py bdist_wheel for json-logging-py: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/0d/2e/1c/c638b7589610d8b9358a6e5eb008edacb8b3e9b6d1edc9479f
  Running setup.py bdist_wheel for olefile: started
  Running setup.py bdist_wheel for olefile: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/c4/19/76/61fc7929d808e51567aff23036ca5fe6ba8336ad0559ca6a27
  Running setup.py bdist_wheel for itsdangerous: started
  Running setup.py bdist_wheel for itsdangerous: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/2c/4a/61/5599631c1554768c6290b08c02c72d7317910374ca602ff1e5
Successfully built configparser json-log

In [15]:
!docker push $image_name

The push refers to repository [docker.io/fboylu/mlaksdep]

fbce4d5f: Preparing 
a7ee889e: Preparing 
b7ec3a5e: Preparing 
6c2f54e3: Preparing 
8b159921: Preparing 
02b8bca3: Preparing 
dd00b1a4: Preparing 
c3c04cbd: Preparing 
daf493f1: Preparing 
c3c04cbd: Waiting g 
7ee889e: Pushing   1.42GB/1.98GBBntu 7ee889e: Pushed   2.049GB/1.98GBlatest: digest: sha256:56bab96a1c923136d6a031bc9cba7c2c700a3491c30aa37243bafd0f95696d09 size: 2618


In [14]:
print('Docker image name {}'.format(image_name))

Docker image name fboylu/mlaksdep
